This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (
    Box,
    MakeVectorVariable,
    MultibodyPlant,
    Parser,
    SinCos,
    Substitute,
    ToLatex,
)
from pydrake.examples import AcrobotPlant, PendulumPlant

from underactuated import ConfigureParser
from underactuated.scenarios import AddShape

# Verifying trigonometric polynomial systems in implicit form.

In order to achieve region of attraction analysis of general MultibodyPlant systems, we lean into the Drake's symbolic engine. Here is an example of the type of symbolic substitutions that go on behind the scenes in the `RegionOfAttraction` code.

In [ ]:
def PrintImplicitPolynomialDynamics(plant, context, sincos_indices):
    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()
    sym_context.SetTimeStateAndParametersFrom(context)
    sym_plant.FixInputPortsFrom(plant, context, sym_context)

    state = sym_context.get_continuous_state()
    derivatives = sym_context.Clone().get_mutable_continuous_state()

    q = MakeVectorVariable(state.num_q(), "q")
    v = MakeVectorVariable(state.num_v(), "v")
    qd = MakeVectorVariable(state.num_q(), "\dot{q}")
    vd = MakeVectorVariable(state.num_v(), "\dot{v}")

    state.SetFromVector(np.hstack((q, v)))
    derivatives.SetFromVector(np.hstack((qd, vd)))
    residual = sym_plant.CalcImplicitTimeDerivativesResidual(sym_context, derivatives)

    s = MakeVectorVariable(state.num_q(), "s")
    c = MakeVectorVariable(state.num_q(), "c")
    subs = {}
    for i in sincos_indices:
        subs[q[i]] = SinCos(s[i], c[i])
    dynamics = Substitute(residual, subs)

    display(Markdown("$" + ToLatex(dynamics, 2) + "= 0$"))


def TestPendulum():
    plant = PendulumPlant()
    context = plant.CreateDefaultContext()
    plant.get_input_port().FixValue(context, [0])
    indices = [0]
    PrintImplicitPolynomialDynamics(plant, context, indices)


TestPendulum()


def TestAcrobot():
    plant = AcrobotPlant()
    context = plant.CreateDefaultContext()
    plant.get_input_port().FixValue(context, [0])
    indices = [0, 1]
    PrintImplicitPolynomialDynamics(plant, context, indices)


TestAcrobot()


def TestCartPole():
    plant = MultibodyPlant(0.0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://underactuated/models/cartpole.urdf")
    plant.Finalize()
    context = plant.CreateDefaultContext()
    plant.get_actuation_input_port().FixValue(context, [0])
    indices = [1]
    PrintImplicitPolynomialDynamics(plant, context, indices)


TestCartPole()


# Example w/ quaternions
def TestFloatingBaseBody():
    plant = MultibodyPlant(0.0)
    AddShape(plant, Box(1, 2, 3), "body", 1)
    plant.Finalize()
    context = plant.CreateDefaultContext()
    indices = []
    PrintImplicitPolynomialDynamics(plant, context, indices)


TestFloatingBaseBody()